# C33 UMAP Visualisation of 18M 

UMAP for 18M

## Imports

In [1]:
import sys 
import os
import pathlib
import time
print(sys.version)

3.6.9 (default, Jul  3 2019, 15:36:16) 
[GCC 5.4.0 20160609]


In [2]:
utils_path = pathlib.Path(os.getcwd() + '/utils')  # i suspect this one is not needed
print(utils_path.exists())
print(os.getcwd())
#sys.path.append(str(utils_path))  # may not be necessary
#sys.path.append(os.getcwd())  # i thnk this is the one that works 
sys.path.append('../') # this one is one level up so we can see the utils lib
print(sys.path)

True
/home/mutaz/ilab1
['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/home/mutaz/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/home/mutaz/.local/lib/python3.6/site-packages/IPython/extensions', '/home/mutaz/.ipython', '../']


In [3]:
import numpy as np
import sklearn
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from utils.data import Data
from utils.config import Config

In [4]:
import umap
import numba

## Read Data

In [5]:
d = Data()
df = d.get18M_features(descriptors='C33')

## need to add commenserate and DFT here 

100%|██████████| 38/38 [09:21<00:00, 14.76s/it]


pd.concat 38 df chunks: 570.41s
set index: 571.08s
time to load 571.08s


In [6]:
df.shape

(18834453, 104)

In [7]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 18834453 entries, Cr1O2_Zr1Cl4 to C1I2_Tl2
Columns: 104 entries, bilayer to SG164
dtypes: float32(98), int64(3), object(3)
memory usage: 12.1 GB


# UMAP

## umap_fit function

In [8]:
# call with df_features[feature_cols]
# labels_df has two columns C33 and communsurate 

def umap_fit(df, labels_df, 
             plot_df_filename, 
             index_name='uid', 
             n_neighbors = 15, 
             n_components = 2, 
             min_dist = 0.1, 
             metric = 'euclidean'):
    
    reducer = umap.UMAP(n_neighbors=n_neighbors,
                        n_components=n_components,
                        min_dist=min_dist,
                        metric=metric,
                        random_state=50
                       )
    
    embeddings = reducer.fit_transform(df)
    embeddings_df = pd.DataFrame(embeddings, columns={'x','y'})
    label1_name = labels_df.columns[0]
    label2_name = labels_df.columns[1]
    
    plot_df = pd.concat([embeddings_df, labels_df], 
              axis=1, 
              ignore_index=False)
    
    plot_df.set_index(index_name, inplace=True)
    
    plot_df.to_csv(Config().get_datapath(plot_df_filename))
    
    return(plot_df)

# Generate Umap plot dfs 

In [9]:
n = df.shape[0]
neighbors_list = [30]
components = 2

filename_pattern = 'umap_18Mdf_{}_{}_{}_{}.csv'

In [ ]:
for neighbors in neighbors_list: 
    time_start = time.time()
    filename = filename_pattern.format('C33',n,neighbors, components)
    print(f'prep for filename: {filename}')
    plot_df = umap_fit(df=df,
                   labels_df=df[['C33','IE']].reset_index(), 
                   plot_df_filename=filename,
                   n_neighbors=neighbors, 
                   n_components=components)
    print('for neighbors: {}. {:.3f} secs'.format(neighbors,time.time()-time_start))

prep for filename: umap_18Mdf_C33_18834453_30_2.csv
